In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*- 

In [2]:
import sqlite3
import re

In [59]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


K = []
L = []
KU = []
L1 = []

In [60]:
# pagrindinis link'as, naudojant scrapinimui einant per pasiūlymų puslapius
burl = "http://www.alio.lt/nekilnojamas-turtas/butai/parduoda.html?page="

# # BeautifulSoup standartinis puslapio duomenų paruošimas
# m = requests.get(burl+"1")
# d = m.content
# soup = BeautifulSoup(d, "html.parser")

for i in range(1,4):
    c = requests.get(burl+str(i))
    d = c.content
    soup = BeautifulSoup(d, "html.parser")
    a = soup.find_all("div", {"class":"price"}) # Kaina
    b = soup.find_all("div", {"class":"description"}) # Kamb.skaičius, Plotis ir Metai
    l = soup.find_all("a", {"class":"vertiselink cursor-pointer"}) #Url link'as
    

# atitinakamų duomenų suradimas ir pridėjimas prie atitinkamo list'o
    for oi in a:
        K.append(int(oi.find("span", {"class":"main_price"}).text.replace(" ", "").replace("€","")))
        
    for op in b:
        dc = []
        if len(op.text.split("|")) == 3:
            ab = op.text.split("|")
            ab = ab[:-1]
            print("ab", ab)
            for k in ab:
                kb = k.split(" ")
                o = list(filter(None, kb))
                dc.append(o[0])
            dc.append(0)
            L.append(dc)
        else:
            ab = op.text.split("|")
            ab = ab[:-1]
            #print(ab)
            for k in ab:
                kb = k.split(" ")
                o = list(filter(None, kb))
                dc.append(o[0])
            L.append(dc)
    
    for h1 in l:
        if h1["href"] not in L1:
            L1.append(h1['href'])

    #pridedama reikiamus list'ų duomenis į dict, kad būtų galima vėliau panaudoti pandas 
    for p,n,q in zip(K,L,L1):
        dic = {}
        dic["Kaina"] = p
        #print(dic["Kaina"])
        dic["Plotas"] = n[0]
        #print(dic["Metai"])
        dic["Skaičius"] = n[1]
        #print(dic["Plotas"])
        dic["Metai"] = n[2]
        #print(dic["Skaičius"])
        dic["Url"] = q
        KU.append(dic)

print(KU)

ab ['33 m² ', '  1 kamb. ']
ab ['36.83 m² ', '  1 kamb. ']
[{'Skaičius': '2', 'Url': 'https://www.alio.lt/skelbimai/2-kamb-butas-klaipeda/ID57874722.html', 'Plotas': '43', 'Metai': '1967', 'Kaina': 48000}, {'Skaičius': '2', 'Url': 'https://www.alio.lt/skelbimai/2-kamb-butas-panevezys-panevezys-zemaiciu-g/ID57864398.html', 'Plotas': '50', 'Metai': '1974', 'Kaina': 31000}, {'Skaičius': '2', 'Url': 'https://www.alio.lt/skelbimai/2-kamb-butas-radviliskio-r-sav-radviliskis-gedimino-g/ID57798338.html', 'Plotas': '41.62', 'Metai': '1959', 'Kaina': 16000}, {'Skaičius': '2', 'Url': 'https://www.alio.lt/skelbimai/2-kamb-butas-telsiu-r-sav-telsiai-vilniaus-g/ID57799074.html', 'Plotas': '50', 'Metai': '1983', 'Kaina': 25000}, {'Skaičius': '3', 'Url': 'https://www.alio.lt/skelbimai/3-kamb-butas-alytus-alytus-vingio-g/ID57789738.html', 'Plotas': '65', 'Metai': '1970', 'Kaina': 27500}, {'Skaičius': '1', 'Url': 'https://www.alio.lt/skelbimai/1-kamb-butas-vilnius-verkiai-verkiu-g/ID57766156.html', 'Plo

In [72]:
# kad pandas rodytų pilną Url link'ą
pd.set_option('display.max_colwidth-1)
', 
# print(KU[0])
# išsaugojame duomenis į SQL duomenų bazę
conn = sqlite3.connect("Alio.db")
cur = conn.cursor()
cur.execute("CREATE TABLE IF NOT EXISTS Butai (Kaina INTERGER, Metai INTERGER, Plotas INTERGER, Skaičius INTERGER, Url TEXT)")

for v in KU:
    cur.execute("INSERT INTO Butai (Kaina, Metai, Plotas, Skaičius, Url) VALUES (?,?,?,?,?)", 
               (v["Kaina"], v["Metai"], v["Plotas"], v["Skaičius"], v["Url"]))

conn.commit()
conn.close()

In [62]:
# sukuriam pandas dataframe ir atrenkam reikiamus duomenis: pig. ir brang. kainą bei url link'ą, kad būtų galima iš kartą patikrinti
df = pd.DataFrame(KU)

print("Pigiausias variantas "+str(df['Kaina'].min())+" "+str(list(df.loc[df['Kaina'].idxmin()])[-1]))
print("Brangusias variantas "+str(df["Kaina"].max())+" "+str(list(df.loc[df['Kaina'].idxmax()])[-1]))

Pigiausias variantas 5400 https://www.alio.lt/skelbimai/1-kamb-butas-siauliai-siauliai-tilzes-g/ID57649778.html
Brangusias variantas 290000 https://www.alio.lt/skelbimai/5-kamb-butas-palanga-palanga-mokyklos-g/ID55015985.html
